In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [4]:
root = '../../resources'

In [5]:
jeff = pd.read_csv(f'{root}/2020/2020-02-02.csv')
jeff.head(5)

,number,time,total,available
0,HE12,2020-02-02 23:56:00,91,57
1,HLM,2020-02-02 23:56:00,583,483
2,RHM,2020-02-02 23:56:00,322,175
3,BM29,2020-02-02 23:56:00,97,93
4,Q81,2020-02-02 23:56:00,96,73


In [11]:
jeff.assign
cp = jeff.groupby('time').mean()
cp

,total,available
time,,
2020-02-02 00:01:00,350.518121,164.158389
2020-02-02 00:06:00,350.518121,164.158389
2020-02-02 00:11:00,350.518121,164.158389
2020-02-02 00:16:00,350.518121,164.158389
2020-02-02 00:21:00,350.518121,164.106040
...,...,...
2020-02-02 23:36:00,350.518121,157.655034
2020-02-02 23:41:00,350.516779,157.267114
2020-02-02 23:46:00,350.518121,156.924832


In [ ]:
def mean_year(year):
    td = timedelta(days=1)
    dt = datetime.strptime(f'{year}-01-01', '%Y-%m-%d')
    dend = datetime.strptime(f'{year}-12-31', '%Y-%m-%d')
    while dt <= dend:
        name = f'{dt.strftime('%Y-%m-%d')}.csv'
        jeff = pd.read_csv(f'{root}/{year}/{name}')
        dt += td

mean_year('2020')